In [1]:
import boto3
import csv
import xmltodict
import datetime

import pandas as pd
import numpy as np

## Get results from Mturk

In [2]:
with open('lia_rootkey.csv') as f: # <-- Change accordingly!
    reader = csv.reader(f)
    rows = [r for r in reader]

ACCESS_KEY = rows[0][0].split('=')[1]
SECRET_KEY = rows[1][0].split('=')[1]

In [3]:
# connect to Mturk
mturk = boto3.client('mturk',
   aws_access_key_id = ACCESS_KEY,
   aws_secret_access_key = SECRET_KEY,
   region_name='us-east-1'
)

In [4]:
# Make sure the connection is ok
print("I have $" + mturk.get_account_balance()['AvailableBalance'] + " in my Mturk account")

I have $2137.14 in my Mturk account


In [5]:
# Retrieve HIT results
hit_id = '31KPKEKW4AEQJPER1N8L7J4KJG9B0F' # <-- Change accordingly!
hit = mturk.get_hit(HITId=hit_id)
print('Hit {} status: {}'.format(hit_id, hit['HIT']['HITStatus']))

Hit 31KPKEKW4AEQJPER1N8L7J4KJG9B0F status: Reviewable


In [6]:
# specify the correct AssignmentStatus: 'Submitted'|'Approved'|'Rejected'
worker_results = mturk.list_assignments_for_hit(HITId=hit_id, AssignmentStatuses=['Submitted'], MaxResults=50)

In [7]:
results = {
    'id': [],
    'accepted': [],
    'submitted': [],
    'elapsed': [],
    'code': [],
    'assignment_id': []
}
for assignment in worker_results['Assignments']:
    results['id'].append(str(assignment['WorkerId']))
    results['accepted'].append(assignment['AcceptTime'])
    results['submitted'].append(assignment['SubmitTime'])
    
    elapsed = assignment['SubmitTime'] - assignment['AcceptTime']
    results['elapsed'].append(round(elapsed.total_seconds() / 60, 2))
    
    answer_dict = xmltodict.parse(assignment['Answer'])
    results['code'].append(str(answer_dict['QuestionFormAnswers']['Answer']['FreeText']))
    
    results['assignment_id'].append(str(assignment['AssignmentId']))

In [8]:
mturk_df = pd.DataFrame(results)

In [10]:
print(mturk_df.shape)
mturk_df.head()

(47, 6)


,id,accepted,submitted,elapsed,code,assignment_id
0,A2RHE12CJSKCRE,2020-06-03 14:36:00+02:00,2020-06-03 14:40:10+02:00,4.17,EMOTIONALITY-FOR-CHATBOTS,33C7UALJVLZ9A0ZWKXXDJ6TK4O1180
1,A39PRBPSKA7A49,2020-06-03 14:36:02+02:00,2020-06-03 14:40:45+02:00,4.72,EMOTIONALITY-FOR-CHATBOTS,338JKRMM260YXZDPJKKC7KFMB1WHAI
2,ACO21NZBM9EGS,2020-06-03 14:36:04+02:00,2020-06-03 14:41:15+02:00,5.18,EMOTIONALITY-FOR-CHATBOTS,3R08VXYT7CWYBNAG7GUSYWNJPH3W76
3,A3F7IDKJWW8962,2020-06-03 14:35:48+02:00,2020-06-03 14:41:52+02:00,6.07,EMOTIONALITY-FOR-CHATBOTS,3KV0LJBBH2MCOU0HCSMBUFCSSRJRM9
4,A2K6CBNIXLVRSS,2020-06-03 14:35:57+02:00,2020-06-03 14:42:02+02:00,6.08,EMOTIONALITY-FOR-CHATBOTS,3YDTZAI2WXH8RZCU8U702UDF22541W


## Pay bonuses
See documentation [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/mturk.html?send_bonus#MTurk.Client.send_bonus).

Note that in the example below I just copy the original dataframe as a `bonus_df` dataframe, but in reality I apply some selection rules and then copy only the selected workers.

In [11]:
bonus_df = mturk_df[['id', 'assignment_id']].copy()

In [12]:
bonus_df.shape

(47, 2)

In [13]:
# Make sure the connection is ok
print("I have $" + mturk.get_account_balance()['AvailableBalance'] + " in my Mturk account")

I have $2137.14 in my Mturk account


In [14]:
worker_ids = bonus_df['id'].values.astype(str)
assignment_ids = bonus_df['assignment_id'].values.astype(str)

In [ ]:
reason = "\
Your responses to at least 3 out of 5 tricky questions included in our survey were consistent, \
which shows you were paying due attention while completing the HIT."

The cell below runs the bonus payment process. I commented the code to make sure I don't run it by chance.

In [ ]:
# for worker_id, assignment_id in zip(worker_ids, assignment_ids):
#     response = mturk.send_bonus(
#         WorkerId=worker_id,
#         BonusAmount='0.25', # <-- Change accordingly!
#         AssignmentId=assignment_id,
#         Reason=reason,
#         UniqueRequestToken=worker_id+'-bonus-01'
#     )
#     print(response)
#     print()

In [ ]:
# check that the account was charged
print("I have $" + mturk.get_account_balance()['AvailableBalance'] + " in my Mturk account")